In [22]:
# Importing all the important libraries
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle 
import argparse

# Setting all the hyperparamters
block_size = 8
batch_size = 4
max_iters = 3000
learning_rate = 3e-4
eval_iters = 50
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
# This is to read the text from the book
with open('pride_and_prejudice.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

# print(len(text))  # Length of the text
# print(text[:100]) # first 100 characters
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '1', '2', '3', '4', '5', '6', '8', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ê']
77


In [15]:
# This is the code for the character-level tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_s = encode("hello")
decoded_s = decode(encoded_s)
print("\" " + str(decoded_s) + " \" : " + str(encoded_s))

" hello ? " : [56, 53, 60, 60, 63, 1, 21]


In [19]:
# Better to use a PyTorch Tensors
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([37, 38, 30, 25, 26,  1,  4,  1, 37, 38, 26, 31, 41, 25, 30, 24, 26, 11,
         0,  0,  0,  0,  0, 24, 29, 22, 37, 40, 26, 38,  1, 30, 11,  0,  0,  0,
        30, 68,  1, 57, 67,  1, 49,  1, 68, 66, 69, 68, 56,  1, 69, 62, 57, 70,
        53, 66, 67, 49, 60, 60, 73,  1, 49, 51, 59, 62, 63, 71, 60, 53, 52, 55,
        53, 52,  9,  1, 68, 56, 49, 68,  1, 49,  1, 67, 57, 62, 55, 60, 53,  1,
        61, 49, 62,  1, 57, 62,  1, 64, 63, 67])


In [ ]:
# Dividing the dataset into train and validation
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)

    return x, y